In [ ]:
#|default_exp sync

# sync

> Propagating small changes in the library back to notebooks
- order: 6

The library is primarily developed in notebooks so any big changes should be made there. But sometimes, it's easier to fix small bugs or typos in the modules directly. `nbdev_update_lib` is the function that will propagate those changes back to the corresponding notebooks. Note that you can't create new cells or reorder cells with that functionality, so your corrections should remain limited.

In [ ]:
#|export
from nbdev.imports import *
from nbdev.config import *
from nbdev.maker import *
from nbdev.process import *
from nbdev.export import *

from execnb.nbio import *
from fastcore.script import *
from fastcore.xtras import *

import ast,tempfile
from importlib import import_module

In [ ]:
#|hide
from pdb import set_trace
from fastcore.test import *

In [ ]:
#|export
def absolute_import(name, fname, level):
    "Unwarps a relative import in `name` according to `fname`"
    if not level: return name
    mods = fname.split(os.path.sep)
    if not name: return '.'.join(mods)
    return '.'.join(mods[:len(mods)-level+1]) + f".{name}"

In [ ]:
test_eq(absolute_import('xyz', 'nbdev', 0), 'xyz')
test_eq(absolute_import('', 'nbdev', 1), 'nbdev')
test_eq(absolute_import('core', 'nbdev', 1), 'nbdev.core')
test_eq(absolute_import('core', 'nbdev/vision', 2), 'nbdev.core')
test_eq(absolute_import('transform', 'nbdev/vision', 1), 'nbdev.vision.transform')
test_eq(absolute_import('notebook.core', 'nbdev/data', 2), 'nbdev.notebook.core')

In [ ]:
#|export
_re_import = re.compile("from\s+\S+\s+import\s+\S")

In [ ]:
#|hide
assert _re_import.match('from foo import bar')
assert not _re_import.match('#from foo import bar')

In [ ]:
#|export
def _to_absolute(code, lib_name):
    if not _re_import.search(code): return code
    res = update_import(code, ast.parse(code).body, lib_name, absolute_import)
    return ''.join(res) if res else code

def _update_lib(nbname, nb_locs, lib_name=None):
    if lib_name is None: lib_name = get_config().lib_name
    absnm = get_config().path('lib_path')/nbname
    nbp = NBProcessor(absnm, ExportModuleProc(), rm_directives=False)
    nbp.process()
    nb = nbp.nb

    for name,idx,code in nb_locs:
        assert name==nbname
        cell = nb.cells[int(idx)]
        directives = ''.join(cell.source.splitlines(True)[:len(cell.directives_)])
        cell.source = directives + _to_absolute(code, lib_name)
    write_nb(nb, absnm)

In [ ]:
#|export
@functools.lru_cache(maxsize=None)
def _mod_files():
    mdir = get_config().path('lib_path').parent
    midx = import_module(f'{get_config().lib_name}._modidx')
    return L(files for mod in midx.d['syms'].values() for _,files in mod.values()).unique()

In [ ]:
#|hide
assert min(_mod_files().map(lambda x: x.endswith('.py'))) is True

In [ ]:
#|export
def _get_call(s):
    top,*rest = s.splitlines()
    return (*top.split(),'\n'.join(rest))

def _script2notebook(fname:str):
    code_cells = Path(fname).read_text().split("\n# %% ")[1:]
    locs = L(_get_call(s) for s in code_cells if not s.startswith('auto '))
    for nbname,nb_locs in groupby(locs, 0).items(): _update_lib(nbname, nb_locs)

@call_parse
def nbdev_update(fname:str=None): # A Python file name to update
    "Propagate change in modules matching `fname` to notebooks that created them"
    if fname and fname.endswith('.ipynb'): raise ValueError("`nbdev_update` operates on .py files.  If you wish to convert notebooks instead, see `nbdev_export`.")
    if os.environ.get('IN_TEST',0): return
    fname = Path(fname or get_config().path('lib_path'))
    lib_dir = get_config().path("lib_path").parent
    files = globtastic(fname, file_glob='*.py').filter(lambda x: str(Path(x).absolute().relative_to(lib_dir) in _mod_files()))
    files.map(_script2notebook)


In [ ]:
#|hide
# nbdev_update("../nbdev/sync.py")

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()